In [20]:
import numpy as np
import pandas as pd
import json
import sys
import os
import matplotlib
matplotlib.use('Agg') 
import matplotlib.pyplot as plt
import seaborn as sns
import pdb

import data_utils
from data_utils import GestureListUtils

#print('appending to sys path')
#print(os.path.dirname(os.path.dirname(os.path.abspath(__file__))))
#sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath(__file__))))

from create_easy_gestures import EasyGestureGenerator

%pylab inline
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'Blues'

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

json_file = '../conv_lstm/torch/final_experiments/diff_input_types/openface_pose_landmark_vel_only/pose_threshold_context_embedding/multi_scale_conv_lstm/checkpoint_40.json'
FDIR = os.path.dirname(json_file)
NUM_CLASSIFY = 5

Populating the interactive namespace from numpy and matplotlib
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
FLAGS = {}
FLAGS['h5_dir'] = '../data_cardiff/openface_h5'
FLAGS['gest_list_h5'] = '../data_cardiff/gest_list/video_type_2/org_gest_list.h5'
FLAGS['cpm_h5_dir'] = '../openface_data/cpm_output'
FLAGS['zface_h5_dir'] = '../data_zface/filtered_headpose'

FLAGS_VIZ_SIGMA_FILTERS = True
FLAGS_VIZ_COMPARE_GESTURES = False

In [22]:
save_easy_gest_list = '044_easy_gest_list.h5'
easy_gest_gen = EasyGestureGenerator(FLAGS['h5_dir'],
                                     FLAGS['cpm_h5_dir'],
                                     FLAGS['zface_h5_dir'],
                                     FLAGS['gest_list_h5'],
                                     #save_name=save_easy_gest_list
    )

# easy_gest_gen.create_gest_list()
'''
for label in range(11):
    gest_mean_vel = easy_gest_gen.get_stats_for_gest('044_static.mp4.txt.h5', label)
    if gest_mean_vel is None:
        continue

    plt.scatter(range(len(gest_mean_vel['nosetip_x'])),
                gest_mean_vel['nosetip_x'])
    plt.scatter(range(len(gest_mean_vel['nosetip_y'])), 
                gest_mean_vel['nosetip_y'],
                color='r')
                #marker='+'
    plt.title('File 044, label: {}'.format(label))
    plt.show()
'''

"\nfor label in range(11):\n    gest_mean_vel = easy_gest_gen.get_stats_for_gest('044_static.mp4.txt.h5', label)\n    if gest_mean_vel is None:\n        continue\n\n    plt.scatter(range(len(gest_mean_vel['nosetip_x'])),\n                gest_mean_vel['nosetip_x'])\n    plt.scatter(range(len(gest_mean_vel['nosetip_y'])), \n                gest_mean_vel['nosetip_y'],\n                color='r')\n                #marker='+'\n    plt.title('File 044, label: {}'.format(label))\n    plt.show()\n"

In [23]:
def visualize_sigma_filters(easy_gest_gen, user, sigma_factor=1):
    new_gest_list, removed_gest_list = easy_gest_gen.create_sigma_gest_list(
        sigma_factor=sigma_factor)
    org_gest_list = GestureListUtils.read_gesture_list(FLAGS['gest_list_h5'])

    user_2 = '040_static.mp4.txt.h5'
    group = 'train' if user in new_gest_list['train'].keys() else 'test'
    group_2 = 'train' if user in new_gest_list['train'].keys() else 'test'
    for label in xrange(11):
        valid_gests = new_gest_list[group][user][str(label)]
        invalid_gests = removed_gest_list[group][user][str(label)]
        
        gest_stats = easy_gest_gen.get_stats_for_gest(user, label)
        old_gest_list = org_gest_list[group][user][str(label)]
        
        valid_nosetip_x, valid_nosetip_y = [], []
        if valid_gests is not None:
            # There are not a lot of gestures O(n^2) is fine here
            
            for i in xrange(len(valid_gests)):
                for j in xrange(len(old_gest_list)):
                    if valid_gests[i][0] == old_gest_list[j][0]:  # Same start frame
                        valid_nosetip_x.append(gest_stats['nosetip_x'][j])
                        valid_nosetip_y.append(gest_stats['nosetip_y'][j])
                        break
                assert len(valid_nosetip_x) == i+1, "Did not find frames for invalid gests"

        
        invalid_nosetip_x, invalid_nosetip_y = [], []
        if invalid_gests is not None:
            for i in xrange(len(invalid_gests)):
                for j in xrange(len(old_gest_list)):
                    if invalid_gests[i][0] == old_gest_list[j][0]: # Same start frame
                        invalid_nosetip_x.append(gest_stats['nosetip_x'][j])
                        invalid_nosetip_y.append(gest_stats['nosetip_y'][j])
                        break

                assert len(invalid_nosetip_x) == i+1, "Did not find frames for invalid gests"
        
        # Visualize
        plt.scatter(range(len(valid_nosetip_x)), valid_nosetip_x, color='b')
        plt.scatter(range(len(valid_nosetip_y)), valid_nosetip_y, color='g')
        plt.scatter(range(len(invalid_nosetip_x)), invalid_nosetip_x, color='r', marker='s')
        plt.scatter(range(len(invalid_nosetip_y)), invalid_nosetip_y, color='c', marker='s')
        plt.title("File {}, label: {}".format(user, label))
        plt.show()
    
    
if FLAGS_VIZ_SIGMA_FILTERS:
    # visualize_sigma_filters(easy_gest_gen, '012_P6_P2_1602_C2.h5', sigma_factor=1)
    
    new_gest_list, removed_gest_list = easy_gest_gen.create_sigma_gest_list(
        sigma_factor=1)
    EasyGestureGenerator.write_gest_list_h5(
        '../data_cardiff/gest_list/video_type_2/1_sigma/1_sigma_gest_list.h5',
        new_gest_list,
        removed_gest_list)
    # print(easy_gest_gen)
    '''
    new_gest_list, removed_gest_list = easy_gest_gen.create_global_gest_list(threshold=1.0, debug=False)
    EasyGestureGenerator.write_gest_list_h5(
        '../data_cardiff/gest_list/video_type_3/threshold_1_0/th_1_0_gest_list.h5',
        new_gest_list,
        removed_gest_list)

    easy_gest_gen.create_global_gest_list()
    '''



Will write easy gesture list to ../data_cardiff/gest_list/video_type_2/1_sigma/1_sigma_gest_list.h5
Did write easy gesture list to ../data_cardiff/gest_list/video_type_2/1_sigma/1_sigma_gest_list.h5
Did save csv at ../data_cardiff/gest_list/video_type_2/1_sigma/1_sigma_gest_list.csv
011_P6_P2_1602_C1.h5: [69, 9, 1, 1, 1, 1, 1, 1, 1, 1, 1]
012_P6_P2_1602_C2.h5: [68, 15, 1, 1, 1, 1, 1, 1, 1, 1, 1]
016_P6_P4_1602_C2.h5: [70, 2, 1, 1, 1, 1, 3, 1, 1, 1, 1]
009_P5_P3_2202_C1.h5: [54, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1]
015_P6_P4_1602_C1.h5: [72, 3, 1, 1, 1, 1, 1, 1, 5, 1, 1]
014_P6_P3_1602_C2.h5: [66, 13, 1, 1, 1, 1, 1, 1, 4, 1, 1]
013_P6_P3_1602_C1.h5: [54, 13, 1, 1, 1, 1, 1, 1, 1, 1, 1]
010_P5_P3_2202_C2.h5: [65, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1]
007_P5_P2_1003_C1.h5: [53, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1]
002_P1_P2_1402_C2.h5: [62, 1, 1, 1, 1, 1, 2, 1, 2, 1, 1]
006_P3_P4_1502_C2.h5: [64, 13, 1, 1, 1, 1, 1, 1, 1, 1, 1]
005_P3_P4_1502_C1.h5: [60, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
001_P1_P2_1402_C1.h5: [57, 2

In [24]:
if FLAGS_VIZ_COMPARE_GESTURES:
    gest_list_data = GestureListUtils.read_gesture_list(FLAGS['gest_list_h5'])
    user1, user2 = '038_static.mp4.txt.h5', '022_static.mp4.txt.h5'
    user3, user4 = '040_static.mp4.txt.h5', '009_static.mp4.txt.h5'
    all_users = [user1, user2, user3, user4]
    all_colors = ['b', 'g', 'r', 'y']
    for label in xrange(11):
        gests = [easy_gest_gen.get_stats_for_gest(user, label) for user in all_users]
        if not gests[0] or not gests[1]: continue
        
        user_nosetip_x, user_nosetip_y = [], []
        for i in xrange(len(all_users)):
            if gests[i] is None: continue
            user_nosetip_x.append([g for g in gests[i]['nosetip_x']])
            user_nosetip_y.append([g for g in gests[i]['nosetip_y']])
        
        for i, u in enumerate(user_nosetip_x):
            plt.scatter(range(len(u)), u, color=all_colors[i])
        plt.title('noseX User {} vs {}, Label: {}'.format(user1, user2, label))
        plt.show()
        
        for i, u in enumerate(user_nosetip_y):
            plt.scatter(range(len(u)), u, color=all_colors[i])
        plt.title('noseY User {} vs {}, Label: {}'.format(user1, user2, label))
        plt.show()
